### Import modules

In [1]:
import psycopg2
import pandas as pd
import numpy as np

### Establish database connection

In [2]:
def connect():
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host = "localhost",
                                database = 'TennisATP',
                                port = 5432,
                                user = 'postgres',
                                password = 'sachin@123')
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    return conn

In [3]:
connect()

Connecting to the PostgreSQL database...


<connection object at 0x00000186F6525E10; dsn: 'user=postgres password=xxx dbname=TennisATP host=localhost port=5432', closed: 0>

### Read csv files

In [4]:
match_stats_df = pd.read_csv("./csv files/Match Stats.csv")
match_stats_df.head(3)

,Tournament_ID,Winner_ID,Loser_ID,Date,Score,Winner_Points,Loser_Points
0,2019-M004,100644,200615,2019-02-25,6-3 6-3,6475.0,439.0
1,2019-0500,100644,104898,2019-06-17,6-4 7-5,4360.0,781.0
2,2019-0414,100644,111797,2019-07-22,6-4 6-2,4325.0,1135.0


In [5]:
player_details_df = pd.read_csv("./csv files/Player Details.csv")
player_details_df.head(3)

,Player_ID,Player_Name,Player_Location,Player_Height,Player_Hand
0,105173,Adrian Mannarino,FRA,183.0,L
1,105077,Albert Ramos,ESP,188.0,L
2,200221,Alejandro Davidovich Fokina,ESP,183.0,R


In [6]:
tournament_stats_df = pd.read_csv("./csv files/Tournament Stats.csv")
tournament_stats_df.head(3)

,tourney_id,w_ace,w_df,w_svpt,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_SvGms,l_bpSaved,l_bpFaced
0,2019-0301,176.0,60.0,1727.0,271.0,77.0,101.0,146.0,75.0,1737.0,265.0,113.0,184.0
1,2019-0308,100.0,36.0,1712.0,265.0,104.0,142.0,76.0,58.0,1713.0,265.0,102.0,194.0
2,2019-0311,292.0,98.0,2309.0,369.0,89.0,119.0,228.0,93.0,2409.0,367.0,134.0,214.0


In [7]:
tournament_details_df = pd.read_csv("./csv files/Tournament_Details.csv")
tournament_details_df.head(3)

,Tournament_ID,Tournament_Name,Surface,Year
0,2020-6932,ATP Rio de Janeiro,Clay,2020
1,2019-M004,Acapulco,Hard,2019
2,2020-M004,Acapulco,Hard,2020


### Create tables

In [13]:
sql_create_table = ["create table player (player_id integer primary key not null, name varchar(50), location varchar(5), height float(5), hand char(2));",
                    "create table tournament (tournament_id varchar(50) primary key not null, name varchar(50), surface varchar(10), year integer);",
                    "create table match_stats (tournament_id varchar(50), winner_id integer, loser_id integer, date varchar(20), score varchar(50), winner_points float(10), loser_points float(10), foreign key (tournament_id) references tournament(tournament_id), foreign key (winner_id) references player(player_id), foreign key (loser_id) references player(player_id));",
                    "create table tournament_stats (tournament_id varchar(50), w_ace float(10), w_df float(10), w_svpt float(10), w_SvGms float(10), w_bpSaved float(10), w_bpFaced float(10), l_ace float(10), l_df float(10), l_svpt float(10), l_SvGms float(10), l_bpSaved float(10), l_bpFaced float(10), foreign key (tournament_id) references tournament(tournament_id));"]

def create_table(conn, sql_q):
    cur = conn.cursor()

    for sql in sql_q:
        cur.execute(sql)

    conn.commit()
    cur.close
    print("Tables created")

In [14]:
conn = connect()
create_table(conn, sql_q = sql_create_table)
conn.close()

Connecting to the PostgreSQL database...
Tables created


In [15]:
sql_insert_table = ["insert into player values (%s, %s, %s, %s, %s)",
                    "insert into tournament values (%s, %s, %s, %s)",
                    "insert into match_stats values (%s, %s, %s, %s, %s, %s, %s)",
                    "insert into tournament_stats values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"]
conn = connect()
cur = conn.cursor()

Connecting to the PostgreSQL database...


### Insert into player table

In [ ]:
# Insert into player table.
player_records = player_details_df.to_numpy().tolist()
player_result = [tuple(r) for r in player_records]
cur.executemany(sql_insert_table[0], player_result)

### Insert into tournament table

In [ ]:
# insert into tournament table.
tournament_records = tournament_details_df.to_numpy().tolist()
tournament_result = [tuple(r) for r in tournament_records]
cur.executemany(sql_insert_table[1], tournament_result)

### Insert into match_stats table

In [ ]:
matchstats_records = match_stats_df.to_numpy().tolist()
matchstats_result = [tuple(r) for r in matchstats_records]
cur.executemany(sql_insert_table[2], matchstats_result)

### Insert into tournament_stats

In [ ]:
tstats_records = tournament_stats_df.to_numpy().tolist()
tstats_result = [tuple(r) for r in tstats_records]
cur.executemany(sql_insert_table[3], tstats_result)

In [ ]:
conn.commit()
cur.close
conn.close()